#### Importação de pacotes

In [1]:
import pandas as pd
import glob
import camaraPy as campy
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [2]:
import importlib
importlib.reload(campy)

<module 'camaraPy' from '/Users/rs116474/Desktop/linha-do-tempo-do-governismo/code/camaraPy.py'>

#### Definição de configurações

In [3]:
pd.options.display.max_rows = 20

#### Ler dados

In [4]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    '''
    
    # Caso o dir_path não possua final em "/", adicone
    if dir_path[-1] != '/':
        dir_path += "/"
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

def read_and_concat(data_, axis=0, sep=None, quotechar=None, ignore_index=True):  
    '''
    >> DESCRIÇÃO
    
    Chama a função find_fpaths definida acima
    e funções built-in do pandas para concatenar
    todos os arquivos em formato .csv que estão
    dentro de um diretório. Retorna um dataframe.
    
    >> PARÂMETROS
    
    data_ -> Uma lista de caminhos para arquivos .csv
    
    axis -> eixo em que vai acontecer a concatenação (default: 0)
    
    sep -> separador dos arquivos .csv em questão (default: ',')
    
    quotechar -> especifica o delimitador que encapsula
    os campos do arquivo (default: None)
    
    ignore_index -> especifica se o índice dos arquivos
    deve ser descartado ao concatenar (default: True)
    '''
    
    # Lê todos os arquivos em uma lista de dataframes
    if quotechar:
        data_ = [ pd.read_csv( file, sep = sep, quotechar = quotechar ) for file in data_ ]
    else:
        data_ = [ pd.read_csv( file, sep = sep ) for file in data_ ]

    
    # Concatena todos em um único dataframe
    data_ = pd.concat(data_, axis = axis, ignore_index = ignore_index)
    
    # Retorna
    return data_

In [5]:
df = []
# Lê apenas os anos entre 2003 e 2018, únicos com dados confiáveis
for year in range(2003, 2019):
    df.extend(
        find_fpaths(dir_path = "../data/csvs/" + str(year) + "/", pattern = "*.csv")
    )

In [6]:
# Concatena tudo em um único dataframe
df = read_and_concat( df, sep = ',' )

In [7]:
# Transforma os identificadores em strings
df["voteId"]       = df.voteId.astype(str)
df["idProposicao"] = df.idProposicao.astype(str)

In [8]:
# Remove colunas com NaN, que ocorrem quando não houve orientação do governo
df = df.dropna( subset = [ "com_governo", "contra_governo" ] )

#### As votações mais consensuais

Queremos descobrir em que ocasiões o governo ou a oposição conseguiu uma maioria significativa.

Para isso, precisamos fazer com que os dados mostrem o total de votos com e contra o governo no agregado de todos os partidos. Por enquanto, eles mostram a divisão por legenda.

In [9]:
df_general = df.groupby(['voteId', 'idProposicao', 'dataHoraFim']).sum().reset_index()

In [10]:
# Calcula dois outros campos, dessa vez mostrando os valores proporcionais de governismo e contra-governismo
df_general['com_governo_%']    = df_general.com_governo    / (df_general.com_governo + df_general.contra_governo)
df_general['contra_governo_%'] = df_general.contra_governo / (df_general.com_governo + df_general.contra_governo)

Abaixo, vamos printar os detalhes sobre as 50 votações onde o governo conseguiu **mais** votos


In [11]:
def print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov):
    
    # Uma request para cada
    vote_data = campy.make_vote_request(vote_id)
    prop_data = campy.make_proposition_request(id_proposicao)

    # Pega os dados
    data_hora_fim = vote_data['dados']['dataHoraFim']
    orientacao_governo = next( (item['voto'] for item in vote_data['dados']['orientacoes'] if item['nomeBancada'] == 'GOV.'), '')
    ementa = vote_data['dados']['proposicao']['ementa']
    titulo = vote_data['dados']['titulo']
    url_proposicao = vote_data['dados']['uriProposicaoPrincipal']
    url_votacao = vote_data['dados']['uri']
    ano = str(prop_data['dados']['ano'])
    numero = str(prop_data['dados']['numero'])
    tipo = prop_data['dados']['siglaTipo']
    
    # Ementa_b é gerada a partir de uma solicitação para o id da proposição.
    # Preciso garantir que ementas original e b são consistentes entre si, já que a api mostrou problemas.
    ementa_b = prop_data['dados']['ementa']
    assert(ementa == ementa_b)
    
    print('\n***')
    print(tipo, numero + '/' + ano)
    print("id_proposicao:", id_proposicao)
    print("ementa:", ementa.strip() )
    print()
    
    print("vote_id:", vote_id)
    print("data_hora_fim:", data_hora_fim)
    print("titulo:", titulo)
    print("orientacao_governo:", orientacao_governo)
    print()
    
    print("votos_pro_gov:", votos_pro_gov)
    print("votos_contra_gov:", votos_contra_gov)
    print()
    
    print("url_proposicao:", url_proposicao)
    print("url_votacao:", url_votacao)
    print('***\n')

In [12]:
top_50_pro_gov = df_general.sort_values(by='com_governo', ascending=False).head(50)

In [13]:
series = [ top_50_pro_gov[ label ].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
PEC 590/2006
id_proposicao: 339287
ementa: Dá nova redação ao parágrafo 1º do art. 58  da Constituição Federal.

NOVA EMENTA: Dá nova redação ao § 1º do art. 58 da Constituição Federal, para garantir a representação proporcional de cada sexo na composição das Mesas e Comissões do Congresso Nacional, da Câmara dos Deputados e do Senado Federal.

vote_id: 6126
data_hora_fim: 2015-03-10 21:42:20.0
titulo: PROPOSTA DE EMENDA À CONSTITUIÇÃO - PRIMEIRO TURNO
orientacao_governo: Sim

votos_pro_gov: 452.0
votos_contra_gov: 3.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/339287
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6126
***


***
PEC 358/2013
id_proposicao: 602633
ementa: Altera os arts. 165 e 166 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

NOVA EMENTA: Altera os arts. 165, 166 e 198 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que esp


***
PEC 358/2013
id_proposicao: 602633
ementa: Altera os arts. 165 e 166 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

NOVA EMENTA: Altera os arts. 165, 166 e 198 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

vote_id: 6089
data_hora_fim: 2015-02-10 21:41:41.0
titulo: DVS - PSOL - ART. 2º DA PEC Nº 358/2013 - 2º TURNO
orientacao_governo: Sim

votos_pro_gov: 427.0
votos_contra_gov: 45.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/602633
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6089
***


***
PEC 333/2004
id_proposicao: 269852
ementa: Modifica a redação do art. 29-A e acrescenta art. 29-B à Constituição Federal para dispor sobre o limite de despesas  e a composição das Câmaras de Vereadores e dá outras providências.

vote_id: 3305
data_hora_fim: 2008-05-27 22:51:27.0
titulo: EMENDA AGLUTINATIVA SUBSTITUTIVA GLOBAL DE Nº 1



***
PLP 366/2013
id_proposicao: 603566
ementa: Altera a Lei Complementar nº 116, de 31 de julho de 2003, que dispõe sobre o Imposto sobre Serviços de Qualquer Natureza; a Lei nº 8.429, de 2 de junho de 1992 (Lei de Improbidade Administrativa); e a Lei Complementar nº 63, de 11 de janeiro de 1990, que dispõe sobre critérios e prazos de crédito das parcelas do produto da arrecadação de impostos de competência dos Estados e de transferências por estes recebidas, pertencentes aos Municípios, e dá outras providências.

vote_id: 6582
data_hora_fim: 2015-09-16 20:28:36.0
titulo: DTQ 5: PT - ART. 4º DO SUBSTITUTIVO
orientacao_governo: Sim

votos_pro_gov: 399.0
votos_contra_gov: 11.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/603566
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6582
***


***
REQ 4282/2016
id_proposicao: 2081484
ementa: Requeremos, nos termos do artigo 155 do Regimento Interno da Câmara dos Deputados, urgência para apreciação do Pr


***
REQ 2409/2015
id_proposicao: 1554118
ementa: Requer, nos termos do art. 155, do Regimento Interno da Câmara dos Deputados, URGÊNCIA para a tramitação de PL nº 5.230, de 2013, que "acrescenta dispositivos à Lei nº 12.592, de 18 de janeiro de 2.012, para dispor sobre a regulamentação e base de tributação do "salão-parceiro" e do "profissional-parceiro".

vote_id: 6525
data_hora_fim: 2015-08-25 20:02:29.0
titulo: URGÊNCIA PARA APRECIAÇÃO DO PL Nº 5.230/2013
orientacao_governo: Sim

votos_pro_gov: 385.0
votos_contra_gov: 32.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/1554118
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6525
***


***
PEC 11/2015
id_proposicao: 1063052
ementa: Altera os arts. 92 e 111-A da Constituição Federal, para explicitar o Tribunal Superior do Trabalho como órgão do Poder Judiciário, alterar os requisitos para o provimento dos cargos de Ministros daquele Tribunal e modificar-lhe a competência.

vote_id: 6874
data_ho

E, agora, os detalhes sobre as 50 votações onde o governo conseguiu **menos** votos


In [14]:
top_50_contra_gov = df_general.sort_values(by='contra_governo', ascending=False).head(50)

In [15]:
series = [ top_50_contra_gov[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)
    


***
PEC 443/2009
id_proposicao: 463155
ementa: Fixa parâmetros para a remuneração dos advogados públicos.

vote_id: 6489
data_hora_fim: 2015-08-06 01:52:16.0
titulo: SUBSTITUTIVO DA COMISSÃO ESPECIAL
orientacao_governo: Não

votos_pro_gov: 16.0
votos_contra_gov: 452.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/463155
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6489
***


***
REQ 9534/2014
id_proposicao: 606381
ementa: Requer a criação de Comissão Externa Temporária para acompanhar as investigações efetuadas pelo Openbaar Ministerie, em Amsterdam, Holanda, acerca das operações e procedimentos no Brasil da empresa SBM Offshore, cuja sede situa-se naquele País, e que envolvem possíveis práticas de suborno.

vote_id: 5775
data_hora_fim: 2014-03-11 20:36:01.0
titulo: REQUER CRIAÇÃO DE COMISSÃO EXTERNA TEMPORÁRIA
orientacao_governo: Não

votos_pro_gov: 28.0
votos_contra_gov: 373.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/prop


***
PL 836/2003
id_proposicao: 112893
ementa: Disciplina o funcionamento de bancos de dados e serviços de proteção ao crédito e congêneres e dá outras providências.

vote_id: 3679
data_hora_fim: 2009-05-19 21:19:57.0
titulo: DVS - PPS - EMENDA Nº 5
orientacao_governo: Não

votos_pro_gov: 32.0
votos_contra_gov: 308.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/112893
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/3679
***


***
PL 5524/2005
id_proposicao: 292328
ementa: Dispõe sobre a instituição de concurso de prognóstico destinado ao desenvolvimento da prática desportiva, a participação de entidades desportivas da modalidade futebol nesse concurso, o parcelamento de débitos tributários e para com o Fundo de Garantia do Tempo de Serviço - FGTS, e dá outras providências
NOVA EMENTA: "Dispõe sobre a instituição de concurso de prognóstico destinado ao desenvolvimento da prática desportiva, a participação de entidades desportivas da modalidade f


***
MPV 778/2017
id_proposicao: 2137734
ementa: Dispõe sobre o parcelamento de débitos junto à Fazenda Nacional relativos às contribuições previdenciárias de responsabilidade dos Estados, do Distrito Federal e dos Municípios.

NOVA EMENTA: Dispõe sobre o parcelamento de débitos com a Fazenda Nacional relativos às contribuições previdenciárias de responsabilidade dos Estados, do Distrito Federal e dos Municípios, e sobre a revisão da dívida previdenciária dos Municípios pelo Poder Executivo federal; altera a Lei nº 9.796, de 5 de maio de 1999; e dá outras providências.

vote_id: 7723
data_hora_fim: 2017-08-22 21:44:10.0
titulo: DTQ 2: PT - EMENDA Nº 25
orientacao_governo: Não

votos_pro_gov: 100.0
votos_contra_gov: 278.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2137734
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/7723
***


***
MPV 635/2013
id_proposicao: 605333
ementa: Dispõe sobre a ampliação do valor do Benefício Garantia-Safra para a


***
PDC 315/2016
id_proposicao: 2077084
ementa: Susta, nos termos do art. 49, V, da Constituição, a vigência do inciso I do art. 3º do Decreto nº 8.616, de 29 de dezembro de 2015, que "Regulamenta o disposto na Lei Complementar nº 148, de 25 de novembro de 2014, e no art. 2º da Lei nº 9.496, de 11 de setembro de 1997."

vote_id: 6834
data_hora_fim: 2016-02-25 19:00:10.0
titulo: REQUERIMENTO DE RETIRADA DE PAUTA
orientacao_governo: Obstrução

votos_pro_gov: 160.0
votos_contra_gov: 245.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2077084
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6834
***


***
MPV 663/2014
id_proposicao: 860923
ementa: Altera a Lei nº 12.096, de 24 de novembro de 2009.

vote_id: 6307
data_hora_fim: 2015-05-12 22:24:45.0
titulo: DVS - PDT - EMENDA Nº 25
orientacao_governo: Não

votos_pro_gov: 199.0
votos_contra_gov: 245.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/860923
url_votacao: https://da


***
MPV 664/2014
id_proposicao: 865929
ementa: Altera as Leis nº 8.213, de 24 de julho de 1991, nº 10.876, de 2 junho de 2004, nº 8.112, de 11 de dezembro de 1990, e a Lei nº 10.666, de 8 de maio de 2003.

NOVA EMENTA: Altera as Leis nºs 8.213, de 24 de julho de 1991, 10.876, de 2 de junho de 2004, 8.112, de 11 de dezembro de 1990, e 10.666, de 8 de maio de 2003; e dá outras providências.

vote_id: 6315
data_hora_fim: 2015-05-13 22:48:21.0
titulo: DVS - PP - ALTERAÇÃO PROMOVIDA PELO ART. 3º  DO PLV
orientacao_governo: Sim

votos_pro_gov: 220.0
votos_contra_gov: 231.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/865929
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6315
***



A maioria das derrotas retumbantes do governo ocorreram sob Dilma. Vamos procurar derrotas em outros períodos.

In [16]:
df_general[ df_general.dataHoraFim < '2007'].sort_values(by='contra_governo', ascending=False)

,voteId,idProposicao,dataHoraFim,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,com_governo_%,contra_governo_%
246,2478,292328,2005-12-13 22:32:48.0,120,NaN,NaN,49.0,303.0,0.139205,0.860795
273,2593,320786,2006-06-07 20:06:48.0,136,NaN,NaN,188.0,250.0,0.429224,0.570776
271,2584,316493,2006-05-17 20:39:28.0,136,NaN,NaN,165.0,240.0,0.407407,0.592593
255,2514,275322,2006-02-07 20:26:27.0,136,NaN,NaN,201.0,223.0,0.474057,0.525943
6,1864,105507,2003-05-06 20:20:09.0,120,120.0,120.0,241.0,217.0,0.526201,0.473799
203,2349,279507,2005-06-07 20:04:06.0,120,NaN,NaN,214.0,203.0,0.513189,0.486811
264,2555,310833,2006-04-04 20:11:00.0,136,NaN,NaN,221.0,188.0,0.540342,0.459658
157,2208,251120,2004-07-07 20:05:16.0,120,NaN,NaN,159.0,188.0,0.458213,0.541787
199,2337,274653,2005-03-30 19:26:30.0,120,NaN,NaN,74.0,186.0,0.284615,0.715385
1,1856,105503,2003-04-15 20:01:34.0,120,120.0,120.0,198.0,185.0,0.516971,0.483029


In [17]:
top_50_contra_lula = df_general[ df_general.dataHoraFim < '2011'].sort_values(by='contra_governo', ascending=False).head(50)

In [18]:
df_general [ (df_general.dataHoraFim > '2003') & (df_general.dataHoraFim < '2007') ].sort_values(by='com_governo')

,voteId,idProposicao,dataHoraFim,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,com_governo_%,contra_governo_%
205,2356,280978,2005-06-16 20:10:31.0,120,NaN,NaN,7.0,141.0,0.047297,0.952703
40,1941,126040,2003-08-28 18:08:04.0,120,120.0,120.0,20.0,34.0,0.370370,0.629630
170,2277,260897,2004-11-17 19:01:23.0,120,NaN,NaN,40.0,48.0,0.454545,0.545455
246,2478,292328,2005-12-13 22:32:48.0,120,NaN,NaN,49.0,303.0,0.139205,0.860795
199,2337,274653,2005-03-30 19:26:30.0,120,NaN,NaN,74.0,186.0,0.284615,0.715385
105,2029,106616,2004-01-22 19:36:33.0,120,120.0,NaN,75.0,38.0,0.663717,0.336283
237,2441,294429,2005-11-01 20:02:49.0,120,NaN,NaN,76.0,75.0,0.503311,0.496689
186,2294,265572,2004-12-08 17:15:58.0,120,NaN,NaN,83.0,85.0,0.494048,0.505952
234,2434,294429,2005-10-20 19:26:20.0,120,NaN,NaN,92.0,36.0,0.718750,0.281250
266,2561,314558,2006-04-12 19:38:05.0,136,NaN,NaN,116.0,173.0,0.401384,0.598616


In [19]:
series = [ top_50_contra_lula[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
PL 2502/2007
id_proposicao: 378478
ementa: Altera a Lei nº 9.478, de 06 de agosto de 1997,que dispõe sobre a política energética nacional, as atividades relativas ao monopólio do petróleo, institui o Conselho Nacional de Política Energética e a Agência Nacional do Petróleo.

vote_id: 4228
data_hora_fim: 2010-03-10 22:07:30.0
titulo: DVS - PPS - EMENDA Nº 387
orientacao_governo: Não

votos_pro_gov: 72.0
votos_contra_gov: 372.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/378478
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/4228
***


***
MPV 475/2009
id_proposicao: 465329
ementa: Dispõe sobre o reajuste dos benefícios mantidos pela Previdência Social em 2010 e 2011.

vote_id: 4318
data_hora_fim: 2010-05-04 21:08:14.0
titulo: DVS - PPS - EMENDA Nº 26
orientacao_governo: Não

votos_pro_gov: 80.0
votos_contra_gov: 327.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/465329
url_votacao: https://dadosabertos.camara.leg.


***
PL 1990/2007
id_proposicao: 366816
ementa: Dispõe sobre o reconhecimento formal das centrais sindicais para os fins que especifica, altera a Consolidação das Leis do Trabalho - CLT, aprovada pelo Decreto-Lei nº 5.452, de 1º de maio de 1943, e dá outras providências.

vote_id: 3062
data_hora_fim: 2007-10-17 23:45:24.0
titulo: DVS EMENDA Nº 21
orientacao_governo: Não

votos_pro_gov: 161.0
votos_contra_gov: 223.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/366816
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/3062
***


***
PL 4776/2005
id_proposicao: 275322
ementa: Dispõe sobre a gestão de florestas públicas para produção sustentável, institui, na estrutura do Ministério do Meio Ambiente, o Serviço Florestal Brasileiro - SFB, cria o Fundo Nacional de Desenvolvimento Florestal - FNDF , e dá outras providências.
NOVA EMENTA DA REDAÇÃO FINAL: Dispõe sobre a gestão de florestas públicas para produção sustentável, institui, na estrutura do Mini


***
MPV 183/2004
id_proposicao: 251120
ementa: Reduz as alíquotas do PIS/PASEP e da COFINS, incidentes na importação e na comercialização do mercado interno de fertilizantes e defensivos agropecuários classificados no Capítulo 31 da NCM, e dá outras providências.
NOVA EMENTA DA REDAÇÃO FINAL: Reduz as alíquotas do PIS/PASEP e da COFINS, incidentes na importação e na comercialização do mercado interno de fertilizantes e defensivos agropecuários classificados e dá outras providências.

vote_id: 2208
data_hora_fim: 2004-07-07 20:05:16.0
titulo: DESTAQUE INCISO 4 DA EMENDA Nº 1
orientacao_governo: Não

votos_pro_gov: 159.0
votos_contra_gov: 188.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/251120
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2208
***


***
MPV 232/2004
id_proposicao: 274653
ementa: Altera a Legislação Tributária Federal e dá outras providências.

vote_id: 2337
data_hora_fim: 2005-03-30 19:26:30.0
titulo: REQUERIMENTO DE RETIRAD


***
MPV 462/2009
id_proposicao: 434563
ementa: Dispõe sobre a prestação de apoio financeiro pela União aos entes federados que recebem recursos do Fundo de Participação dos Municípios - FPM, no exercício de 2009, com o objetivo de superar dificuldades financeiras emergenciais, e dá outras providências.

vote_id: 3800
data_hora_fim: 2009-08-19 20:14:22.0
titulo: DVS - PSDB - ARTIGO 9º DO PLV
orientacao_governo: Não

votos_pro_gov: 193.0
votos_contra_gov: 169.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/434563
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/3800
***


***
MPV 242/2005
id_proposicao: 279507
ementa: Altera dispositivos da Lei nº 8.213, de 24 de julho de 1991, que dispõe sobre os Planos de Benefícios da Previdência Social e dá outras providências.  
NOVA EMENTA DA REDAÇÃO FINAL: Altera dispositivos das Leis nºs 8.212 e  8.213, ambas de 24 de julho de 1991,e dá outras providências.

vote_id: 2351
data_hora_fim: 2005-06-15 20:01:18


***
MPV 281/2006
id_proposicao: 314559
ementa: Reduz a zero as alíquotas de imposto de renda e da Contribuição Provisória sobre Movimentação ou Transmissão de Valores e de Créditos e Direitos de Natureza Financeira (CPMF) nos casos que especifica, e dá outras providências.  
NOVA EMENTA DA REDAÇÃO FINAL:  Reduz a zero as alíquotas do imposto de renda e da Contribuição Provisória sobre Movimentação ou Transmissão de Valores e de Créditos e Direitos de Natureza Financeira - CPMF nos casos que especifica; altera a Lei nº 9.311, de 24 de outubro de 1996; e dá outras providências.

vote_id: 2565
data_hora_fim: 2006-04-26 21:56:57.0
titulo: PARECER DO RELATOR PELA ADMISSIBILIDADE
orientacao_governo: Sim

votos_pro_gov: 190.0
votos_contra_gov: 155.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/314559
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2565
***



#### Votações por partido

Agora vamos analisar como os três principais partidos dessa fase da democracia brasileira se comportaram ao longo dos anos.

Primeiro, o **PT**.

In [20]:
pt = df[ df.siglaPartido == 'PT' ]

De início, vamos olhar para ocasiões em que o PT foi contra o governo, mas durante os mandados de Lula e Dilma. É contraintuitivo, mas ocorreu com alguma frequência.

In [21]:
pt_no_governo = pt[ pt.dataHoraFim < '2016-05-12'].sort_values(by='contra_governo', ascending = False).head(25) # data do afastamento inicial de Dilma no processo de impeachment

In [22]:
series = [ pt_no_governo[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
MPV 148/2003
id_proposicao: 149101
ementa: Atribui competências à Agência Nacional de Saúde Suplementar - ANS e fixa as diretrizes a serem observadas na definição de normas para implantação de programas especiais de incentivo à adaptação de contratos anteriores à Lei nº 9.656, de 3 de junho de 1998.

vote_id: 2108
data_hora_fim: 2004-03-10 19:02:05.0
titulo: REQUERIMENTO DE PREFERÊNCIA PARA MEDIDA PROVISÓRIA -  TEXTO INICIAL
orientacao_governo: Sim

votos_pro_gov: 1.0
votos_contra_gov: 69.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/149101
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2108
***


***
MPV 388/2007
id_proposicao: 366620
ementa: Altera e acresce dispositivos à Lei nº 10.101, de 19 de dezembro de 2000.

vote_id: 3096
data_hora_fim: 2007-11-20 20:07:06.0
titulo: PROJETO DE LEI DE CONVERSÃO Nº 36 DO SENADO FEDERAL
orientacao_governo: Não

votos_pro_gov: 2.0
votos_contra_gov: 68.0

url_proposicao: https://dadosabertos.camara.l


***
PL 2401/2003
id_proposicao: 140375
ementa: Estabelece normas de segurança e mecanismos de fiscalização de atividades que envolvam organismos geneticamente modificados - OGM e seus derivados, cria o Conselho Nacional de Biossegurança - CNBS, reestrutura a Comissão Técnica Nacional de Biossegurança - CTNBio, dispõe sobre a Política Nacional de Biossegurança e dá outras providências.
NOVA EMENTA DA REDAÇÃO FINAL: Regulamenta os incisos II, IV e V do § 1º do art. 225 da Constituição Federal, estabelece normas de segurança e mecanismos de fiscalização de atividades que envolvam organismos geneticamente modificados - OGM e seus derivados, cria o Conselho Nacional de Biossegurança - CNBS, reestrutura a Comissão Técnica Nacional de Biossegurança - CTNBio, dispõe sobre a Política Nacional de Biossegurança - PNB, revoga a Lei n° 8.974, de 5 de janeiro de 1995, e a Medida Provisória nº 2.191-9, de 23 de agosto de 2001, e os arts. 5º, 6º, 7º, 8°, 9º, 10 e 16 da Lei nº 10.814, de 15 de dezembr


***
MPV 207/2004
id_proposicao: 262198
ementa: Altera disposições das Leis nºs 10.683, de 28 de maio de 2003, e 9.650, de 27 de maio de 1998.

vote_id: 2289
data_hora_fim: 2004-12-01 01:17:23.0
titulo: PARECER  DO RELATOR PELA ADMISSIBILIDADE
orientacao_governo: Sim

votos_pro_gov: 52.0
votos_contra_gov: 29.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/262198
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2289
***


***
REQ 10361/2014
id_proposicao: 617857
ementa: Requer urgência, nos termos do art. 155, do Regimento Interno da Câmara dos Deputados, para apreciação do PDC 1.491/2014 que susta a aplicação do Decreto nº 8.243, de 23 de maio de 2014, que institui a Política Nacional de Participação Social - PNPS e o Sistema Nacional de Participação Social - SNPS, e dá outras providências.

vote_id: 5970
data_hora_fim: 2014-07-15 19:59:45.0
titulo: URGÊNCIA PARA APRECIAÇÃO DO PDC Nº 1.491/2014
orientacao_governo: Não

votos_pro_gov: 31.0
votos_c

Também vamos dar uma olhada em espaços de tempo específicos em que notamos picos de rebeldia da base petista no gráfico de área.

In [23]:
pt_2010_e_2011 = pt_no_governo[ 
    (pt_no_governo.dataHoraFim > '2009-12-31') & 
    (pt_no_governo.dataHoraFim < '2012-01-01')
].sort_values(by='contra_governo', ascending=False).head(25)

In [24]:
pt_2014_a_2016 = pt_no_governo[ 
    (pt_no_governo.dataHoraFim > '2013-12-31') & 
    (pt_no_governo.dataHoraFim < '2017-01-01')
].sort_values(by='contra_governo', ascending=False).head(25)

In [25]:
series = [ pt_2014_a_2016[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
PL 4148/2008
id_proposicao: 412728
ementa: Altera e acresce dispositivos à Lei nº 11.105, de 24 de março de 2005.

NOVA EMENTA: Altera a Lei nº 11.105, de 24 de março de 2005.

vote_id: 6259
data_hora_fim: 2015-04-28 22:12:18.0
titulo: SUBEMENDA SUBSTITUTIVA DA CDEIC
orientacao_governo: Sim

votos_pro_gov: 2.0
votos_contra_gov: 58.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/412728
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6259
***


***
PL 7735/2014
id_proposicao: 619150
ementa: Regulamenta o inciso II do § 1º e o § 4º do art. 225 da Constituição; os arts. 1, 8, "j", 10, "c", 15 e 16, §§ 3 e 4 da Convenção sobre Diversidade Biológica, promulgada pelo Decreto nº 2.519, de 16 de março de 1998; dispõe sobre o acesso ao patrimônio genético; sobre a proteção e o acesso ao conhecimento tradicional associado; sobre a repartição de benefícios para conservação e uso sustentável da biodiversidade; e dá outras providências.

NOVA EMENTA: Reg


***
REQ 10361/2014
id_proposicao: 617857
ementa: Requer urgência, nos termos do art. 155, do Regimento Interno da Câmara dos Deputados, para apreciação do PDC 1.491/2014 que susta a aplicação do Decreto nº 8.243, de 23 de maio de 2014, que institui a Política Nacional de Participação Social - PNPS e o Sistema Nacional de Participação Social - SNPS, e dá outras providências.

vote_id: 5970
data_hora_fim: 2014-07-15 19:59:45.0
titulo: URGÊNCIA PARA APRECIAÇÃO DO PDC Nº 1.491/2014
orientacao_governo: Não

votos_pro_gov: 31.0
votos_contra_gov: 29.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/617857
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/5970
***



In [26]:
series = [ pt_2014_a_2016[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
PL 4148/2008
id_proposicao: 412728
ementa: Altera e acresce dispositivos à Lei nº 11.105, de 24 de março de 2005.

NOVA EMENTA: Altera a Lei nº 11.105, de 24 de março de 2005.

vote_id: 6259
data_hora_fim: 2015-04-28 22:12:18.0
titulo: SUBEMENDA SUBSTITUTIVA DA CDEIC
orientacao_governo: Sim

votos_pro_gov: 2.0
votos_contra_gov: 58.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/412728
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6259
***


***
PL 7735/2014
id_proposicao: 619150
ementa: Regulamenta o inciso II do § 1º e o § 4º do art. 225 da Constituição; os arts. 1, 8, "j", 10, "c", 15 e 16, §§ 3 e 4 da Convenção sobre Diversidade Biológica, promulgada pelo Decreto nº 2.519, de 16 de março de 1998; dispõe sobre o acesso ao patrimônio genético; sobre a proteção e o acesso ao conhecimento tradicional associado; sobre a repartição de benefícios para conservação e uso sustentável da biodiversidade; e dá outras providências.

NOVA EMENTA: Reg


***
REQ 10361/2014
id_proposicao: 617857
ementa: Requer urgência, nos termos do art. 155, do Regimento Interno da Câmara dos Deputados, para apreciação do PDC 1.491/2014 que susta a aplicação do Decreto nº 8.243, de 23 de maio de 2014, que institui a Política Nacional de Participação Social - PNPS e o Sistema Nacional de Participação Social - SNPS, e dá outras providências.

vote_id: 5970
data_hora_fim: 2014-07-15 19:59:45.0
titulo: URGÊNCIA PARA APRECIAÇÃO DO PDC Nº 1.491/2014
orientacao_governo: Não

votos_pro_gov: 31.0
votos_contra_gov: 29.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/617857
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/5970
***



Agora, o **PSDB**.

In [27]:
psdb = df[ df.siglaPartido == 'PSDB']

Assim como no caso do PT, é especialmente interessante ver quando os tucanos fizeram algo diferente do que se espera da oposição. Quando foi que o PSDB apoiou o governo de Lula ou Dilma?

In [28]:
psdb_na_oposicao = psdb[ psdb.dataHoraFim < '2016-05-12'].sort_values(by='com_governo', ascending = False).head(100) # data do afastamento inicial de Dilma no processo de impeachment]

In [29]:
series = [ psdb_na_oposicao[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
MPV 345/2007
id_proposicao: 339823
ementa: Dispõe sobre cooperação federativa no âmbito da segurança pública.
NOVA EMENTA DA REDAÇÃO FINAL: Dispõe sobre cooperação federativa no âmbito da segurança pública e revoga a Lei nº 10.277, de 10 de setembro de 2001.

vote_id: 2690
data_hora_fim: 2007-03-06 19:36:06.0
titulo: REQUERIMENTO DE ADIAMENTO DA VOTAÇÃO POR DUAS SESSÕES
orientacao_governo: Não

votos_pro_gov: 57.0
votos_contra_gov: 1.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/339823
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2690
***


***
PL 7225/2006
id_proposicao: 327896
ementa: Altera a Lei nº 7.210, de 11 de julho de 1984,   Lei de Execução Penal, para prever como falta disciplinar grave a utilização de telefone celular pelo preso.
NOVA EMENTA DA REDAÇÃO FINAL: Altera a Lei nº 7.210, de 11 de julho de 1984,   Lei de Execução Penal, e o Decreto-Lei nº 2.848, de 7 de dezembro de 1940 - Código Penal, para prever como falta disci


***
PEC 536/2006
id_proposicao: 320338
ementa: Altera os arts. 159, 239 e 240 da Constituição Federal e acrescenta o art. 227-A a seu texto, para dispor sobre o Fundo de Promoção da Igualdade Racial.

vote_id: 2496
data_hora_fim: 2006-01-24 21:52:35.0
titulo: DVS PFL - EXPRESSÃO: "PARA CADA REGIÃO POLÍTICO-ADMINISTRATIVA"
orientacao_governo: Não

votos_pro_gov: 47.0
votos_contra_gov: 2.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/320338
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2496
***


***
PEC 590/2006
id_proposicao: 339287
ementa: Dá nova redação ao parágrafo 1º do art. 58  da Constituição Federal.

NOVA EMENTA: Dá nova redação ao § 1º do art. 58 da Constituição Federal, para garantir a representação proporcional de cada sexo na composição das Mesas e Comissões do Congresso Nacional, da Câmara dos Deputados e do Senado Federal.

vote_id: 6126
data_hora_fim: 2015-03-10 21:42:20.0
titulo: PROPOSTA DE EMENDA À CONSTITUIÇÃO - PRIMEIRO 


***
PLP 366/2013
id_proposicao: 603566
ementa: Altera a Lei Complementar nº 116, de 31 de julho de 2003, que dispõe sobre o Imposto sobre Serviços de Qualquer Natureza; a Lei nº 8.429, de 2 de junho de 1992 (Lei de Improbidade Administrativa); e a Lei Complementar nº 63, de 11 de janeiro de 1990, que dispõe sobre critérios e prazos de crédito das parcelas do produto da arrecadação de impostos de competência dos Estados e de transferências por estes recebidas, pertencentes aos Municípios, e dá outras providências.

vote_id: 6582
data_hora_fim: 2015-09-16 20:28:36.0
titulo: DTQ 5: PT - ART. 4º DO SUBSTITUTIVO
orientacao_governo: Sim

votos_pro_gov: 44.0
votos_contra_gov: 2.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/603566
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6582
***


***
PLP 124/2015
id_proposicao: 1555340
ementa: Dispõe sobre a aposentadoria compulsória por idade, com proventos proporcionais, nos termos do inciso II do § 1º do 


***
PL 2086/2007
id_proposicao: 368448
ementa: Dispõe sobre o desconto de créditos da Contribuição para o PIS/PASEP e da COFINS, na aquisição no mercado interno ou importação de bens de capital destinados à produção dos bens relacionados nos Anexos I e II da Lei nº 10.485, de 3 de julho de 2002, e dos produtos classificados na Tabela de Incidência do Imposto sobre Produtos Industrializados - TIPI, aprovada pelo Decreto nº 6.006, de 28 de dezembro de 2006; autoriza a concessão de subvenção econômica nas operações de empréstimo e financiamento destinadas às empresas dos setores de calçados e artefatos de couro, têxtil, de confecção e de móveis de madeira; e dá outras providências.

vote_id: 2967
data_hora_fim: 2007-09-25 20:41:21.0
titulo: DVS - EMENDA Nº 21
orientacao_governo: Não

votos_pro_gov: 42.0
votos_contra_gov: 6.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/368448
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2967
***


***
PEC 37/2


***
PEC 395/2014
id_proposicao: 611966
ementa: Altera a redação do inciso IV do art. 206 da Constituição Federal, referente à gratuidade do ensino público em estabelecimentos oficiais.

vote_id: 6819
data_hora_fim: 2016-02-17 21:52:46.0
titulo: DTQ 2: DEM - EXPR.: "E DE MESTRADO PROFISSIONAL", INCISO IV DO ART. 206 DA CF/88
orientacao_governo: Não

votos_pro_gov: 41.0
votos_contra_gov: 4.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/611966
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6819
***


***
MPV 462/2009
id_proposicao: 434563
ementa: Dispõe sobre a prestação de apoio financeiro pela União aos entes federados que recebem recursos do Fundo de Participação dos Municípios - FPM, no exercício de 2009, com o objetivo de superar dificuldades financeiras emergenciais, e dá outras providências.

vote_id: 3798
data_hora_fim: 2009-08-18 18:56:40.0
titulo: DVS - PSDB - ARTIGO 4º DO PLV
orientacao_governo: Não

votos_pro_gov: 41.0
votos_contra_g


***
REQ 4282/2016
id_proposicao: 2081484
ementa: Requeremos, nos termos do artigo 155 do Regimento Interno da Câmara dos Deputados, urgência para apreciação do Projeto de Lei nº 2.646, de 2015, que dispõe sobre o subsídio de Ministro do Supremo Tribunal Federal referido no inciso XV do art. 48 da Constituição Federal.

vote_id: 6943
data_hora_fim: 2016-05-03 21:35:55.0
titulo: URGÊNCIA PARA APRECIAÇÃO DO PL Nº 2.646/2015
orientacao_governo: Sim

votos_pro_gov: 39.0
votos_contra_gov: 5.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2081484
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6943
***


***
PL 4428/2004
id_proposicao: 269767
ementa: Autoriza o Poder Executivo a criar Colégio Militar nas cidades que especifica.

vote_id: 2512
data_hora_fim: 2006-02-07 20:26:27.0
titulo: REQUERIMENTO DE URGÊNCIA
orientacao_governo: Sim

votos_pro_gov: 39.0
votos_contra_gov: 0.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2697


***
REQ 7546/2010
id_proposicao: 489010
ementa: Requerimento, para regime de urgência urgentíssima para tramitação do PL 2944-A, de 2004, que "institui normas sobre jogos de bingo em todo o Território Nacional.

vote_id: 4498
data_hora_fim: 2010-12-08 20:36:45.0
titulo: URGÊNCIA PARA O PL 2.944/2004
orientacao_governo: Sim

votos_pro_gov: 38.0
votos_contra_gov: 9.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/489010
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/4498
***


***
REQ 7772/2013
id_proposicao: 576895
ementa: Requer regime de urgência para apreciação de proposição.

vote_id: 5401
data_hora_fim: 2013-05-21 23:59:38.0
titulo: REQUERIMENTO DE URGÊNCIA PARA O PLP Nº 200/2012
orientacao_governo: Sim

votos_pro_gov: 38.0
votos_contra_gov: 0.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/576895
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/5401
***


***
PL 4148/2008
id_proposicao: 412728
ementa


***
MPV 671/2015
id_proposicao: 1050558
ementa: Institui o Programa de Modernização da Gestão e de Responsabilidade Fiscal do Futebol Brasileiro, dispõe sobre a gestão temerária no âmbito das entidades desportivas profissionais, e dá outras providências.

NOVA EMENTA: Estabelece princípios e práticas de responsabilidade fiscal e financeira e de gestão transparente e democrática para entidades desportivas profissionais de futebol; institui parcelamentos especiais para recuperação de dívidas pela União, cria a Autoridade Pública de Governança do Futebol - APFUT;  dispõe sobre a gestão temerária no âmbito das entidades desportivas profissionais; cria a Loteria Exclusiva - LOTEX; altera as Leis nºs 9.615, de 24 de março de 1998, 8.212, de 24 de julho de 1991, 10.671, de 15 de maio de 2003, 10.891, de 9 de julho de 2004, 11.345, de 14 de setembro de 2006, e 11.438, de 29 de dezembro de 2006, e os Decretos-Leis nºs 3.688, de 3 de outubro de 1941, e 204, de 27 de fevereiro de 1967;  revoga a

E como os tucanos se comportaram nas reformas propostas no primeiro mandato de Lula – aquelas que racharam o PT, a da previdência e a tributária?

In [30]:
# Previdência
psdb[ psdb.voteId.isin(['1932', '1940']) ]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,dataHoraFim,dataHoraInicio,idProposicao,siglaPartido,voteId
709,11,11.0,11.0,37.0,8.0,2003-08-13 23:08:57.0,2003-08-13 18:33:09.0,113716,PSDB,1932
789,11,11.0,11.0,28.0,25.0,2003-08-27 20:01:18.0,2003-08-27 15:02:57.0,113716,PSDB,1940


In [31]:
# Tributária
psdb[ psdb.voteId.isin(['2116', '2121']) ]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,dataHoraFim,dataHoraInicio,idProposicao,siglaPartido,voteId
2999,10,NaN,NaN,40.0,0.0,2004-03-24 14:00:31.0,2004-03-24 09:12:59.0,150410,PSDB,2116
3079,10,NaN,NaN,40.0,0.0,2004-03-31 20:53:32.0,2004-03-31 19:03:52.0,150410,PSDB,2121


E que estranho alinhamento é esse entre PSDB e governo no começo da derrocada de Dilma Rousseff?

In [32]:
psdb_dilma_2 = psdb[ (psdb.dataHoraFim > '2014-12-31') & (psdb.dataHoraFim < '2016-05-12') ].sort_values(by='com_governo', ascending=False).head(25)

In [33]:
df[ (df.dataHoraFim > '2017-10-15') & (df.dataHoraFim < '2017-10-26') ]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,dataHoraFim,dataHoraInicio,idProposicao,siglaPartido,voteId
36804,0,NaN,NaN,2.0,2.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,Avante,7926
36805,1,NaN,NaN,19.0,1.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,DEM,7926
36806,2,NaN,NaN,0.0,11.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PCdoB,7926
36807,3,NaN,NaN,1.0,16.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PDT,7926
36808,4,NaN,NaN,2.0,0.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PEN,7926
36809,5,NaN,NaN,2.0,3.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PHS,7926
36810,6,NaN,NaN,39.0,3.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PMDB,7926
36811,7,NaN,NaN,24.0,3.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PP,7926
36812,8,NaN,NaN,0.0,4.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PPS,7926
36813,9,NaN,NaN,26.0,2.0,2017-10-17 20:31:23.0,2017-10-17 13:55:01.0,2156497,PR,7926


In [34]:
series = [ psdb_dilma_2[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
PEC 358/2013
id_proposicao: 602633
ementa: Altera os arts. 165 e 166 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

NOVA EMENTA: Altera os arts. 165, 166 e 198 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

vote_id: 6088
data_hora_fim: 2015-02-10 21:41:41.0
titulo: PROPOSTA DE EMENDA À CONSTITUIÇÃO - 2º TURNO
orientacao_governo: Sim

votos_pro_gov: 51.0
votos_contra_gov: 0.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/602633
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6088
***


***
PEC 358/2013
id_proposicao: 602633
ementa: Altera os arts. 165 e 166 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

NOVA EMENTA: Altera os arts. 165, 166 e 198 da Constituição Federal, para tornar obrigatória a execução da programação orçamentária que especifica.

vote_id: 6089
data_ho


***
PEC 11/2015
id_proposicao: 1063052
ementa: Altera os arts. 92 e 111-A da Constituição Federal, para explicitar o Tribunal Superior do Trabalho como órgão do Poder Judiciário, alterar os requisitos para o provimento dos cargos de Ministros daquele Tribunal e modificar-lhe a competência.

vote_id: 6829
data_hora_fim: 2016-02-23 21:26:38.0
titulo: PROPOSTA DE EMENDA À CONSTITUIÇÃO - PRIMEIRO TURNO
orientacao_governo: Sim

votos_pro_gov: 44.0
votos_contra_gov: 0.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/1063052
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6829
***


***
PEC 11/2015
id_proposicao: 1063052
ementa: Altera os arts. 92 e 111-A da Constituição Federal, para explicitar o Tribunal Superior do Trabalho como órgão do Poder Judiciário, alterar os requisitos para o provimento dos cargos de Ministros daquele Tribunal e modificar-lhe a competência.

vote_id: 6874
data_hora_fim: 2016-03-22 21:50:24.0
titulo: PROPOSTA DE EMENDA À CONS

E, enfim, o **MDB**:

In [35]:
mdb = df[ (df.siglaPartido == 'PMDB') | (df.siglaPartido == 'MDB')]

O partido é famoso por estar **sempre** na situação. Quando ele fez oposição, afinal?

In [36]:
mdb_oposicao = mdb.sort_values(by='contra_governo', ascending=False).head(25)

In [37]:
series = [ mdb_oposicao[label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
MPV 339/2006
id_proposicao: 339689
ementa: Regulamenta o art. 60 do Ato das Disposições Constitucionais Transitórias e dá outras providências.

vote_id: 2815
data_hora_fim: 2007-05-29 19:59:13.0
titulo: EMENDA Nº 7 COM PARECER PELA REJEIÇÃO
orientacao_governo: Não

votos_pro_gov: 9.0
votos_contra_gov: 72.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/339689
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/2815
***


***
MPV 462/2009
id_proposicao: 434563
ementa: Dispõe sobre a prestação de apoio financeiro pela União aos entes federados que recebem recursos do Fundo de Participação dos Municípios - FPM, no exercício de 2009, com o objetivo de superar dificuldades financeiras emergenciais, e dá outras providências.

vote_id: 3807
data_hora_fim: 2009-08-19 20:14:22.0
titulo: DVS - DEM - EMENDA Nº 26
orientacao_governo: Não

votos_pro_gov: 3.0
votos_contra_gov: 69.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/434563



***
PL 1057/2007
id_proposicao: 351362
ementa: Dispõe sobre o combate a práticas tradicionais nocivas e à proteção dos direitos fundamentais de crianças indígenas, bem como pertencentes a outras sociedades ditas não tradicionais.

vote_id: 6531
data_hora_fim: 2015-08-26 20:13:09.0
titulo: SUBEMENDA SUBSTITUTIVA GLOBAL
orientacao_governo: Não

votos_pro_gov: 2.0
votos_contra_gov: 58.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/351362
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/6531
***


***
PL 6025/2005
id_proposicao: 302638
ementa: Altera o art. 666 da Lei nº 5.869, de 11 de janeiro de 1973 - Código de Processo Civil, para dispor acerca da penhora sobre máquinas, instrumentos e implementos agrícolas.

vote_id: 5757
data_hora_fim: 2014-02-11 21:15:09.0
titulo: DVS - PTB - EMENDA AGLUTINATIVA Nº 7
orientacao_governo: Não

votos_pro_gov: 1.0
votos_contra_gov: 57.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/30263


***
PL 2565/2011
id_proposicao: 524645
ementa: Modifica as Leis nº 9.478, de 6 de agosto de 1997, e nº 12.351, de 22 de dezembro de 2010, para determinar novas regras de distribuição entre os entes da Federação dos royalties e da participação especial devidos em função da exploração de petróleo, gás natural e outros hidrocarbonetos fluidos, e para aprimorar o marco regulatório sobre a exploração desses recursos no regime de partilha.

vote_id: 5122
data_hora_fim: 2012-11-06 20:22:02.0
titulo: PROJETO DE LEI
orientacao_governo: Não

votos_pro_gov: 12.0
votos_contra_gov: 54.0

url_proposicao: https://dadosabertos.camara.leg.br/api/v2/proposicoes/524645
url_votacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/5122
***



Entre elas aparece a MP dos Portos. Temer é acusado de ter influenciado a votação dela para beneficiar algumas empresas. Vamos olhar as votações da proposição:

In [38]:
series = [ mdb[ mdb.idProposicao == '562615'][label].tolist() for label in [ "voteId", "idProposicao", "com_governo", "contra_governo" ] ]
for vote_id, id_proposicao, votos_pro_gov, votos_contra_gov in zip(*series): # * is used to unpack a list of variables
    print_details(vote_id, id_proposicao, votos_pro_gov, votos_contra_gov)


***
MPV 595/2012
id_proposicao: 562615
ementa: Dispõe sobre a exploração direta e indireta, pela União, de portos e instalações portuárias e sobre as atividades desempenhadas pelos operadores portuários, e dá outras providências.

NOVA EMENTA: Dispõe sobre a exploração direta e indireta pela União de portos e instalações portuárias e sobre as atividades desempenhadas pelos operadores portuários; altera as Leis nºs 5.025, de 10 de junho de 1966, 10.233, de 5 de junho de 2001, 10.683, de 28 de maio de 2003, 9.719, de 27 de novembro de 1998, e 8.213, de 24 de julho de 1991; revoga as Leis nºs 8.630, de 25 de fevereiro de 1993, e 11.610, de 12 de dezembro de 2007, e dispositivos das Leis nºs 11.314, de 3 de julho de 2006, e 11.518, de 5 de setembro de 2007; e dá outras providências.

vote_id: 5332
data_hora_fim: 2013-05-08 22:09:25.0
titulo: REQUERIMENTO DE RETIRADA DE PAUTA
orientacao_governo: Não

votos_pro_gov: 65.0
votos_contra_gov: 4.0

url_proposicao: https://dadosabertos.camara.leg


***
MPV 595/2012
id_proposicao: 562615
ementa: Dispõe sobre a exploração direta e indireta, pela União, de portos e instalações portuárias e sobre as atividades desempenhadas pelos operadores portuários, e dá outras providências.

NOVA EMENTA: Dispõe sobre a exploração direta e indireta pela União de portos e instalações portuárias e sobre as atividades desempenhadas pelos operadores portuários; altera as Leis nºs 5.025, de 10 de junho de 1966, 10.233, de 5 de junho de 2001, 10.683, de 28 de maio de 2003, 9.719, de 27 de novembro de 1998, e 8.213, de 24 de julho de 1991; revoga as Leis nºs 8.630, de 25 de fevereiro de 1993, e 11.610, de 12 de dezembro de 2007, e dispositivos das Leis nºs 11.314, de 3 de julho de 2006, e 11.518, de 5 de setembro de 2007; e dá outras providências.

vote_id: 5357
data_hora_fim: 2013-05-15 03:36:25.0
titulo: DVS - DEM - EXPR. "VEDADA A EXCLUSÃO DE ÁREA", CPT DO ART. 15 DO PLV
orientacao_governo: Não

votos_pro_gov: 2.0
votos_contra_gov: 37.0

url_proposic


***
MPV 595/2012
id_proposicao: 562615
ementa: Dispõe sobre a exploração direta e indireta, pela União, de portos e instalações portuárias e sobre as atividades desempenhadas pelos operadores portuários, e dá outras providências.

NOVA EMENTA: Dispõe sobre a exploração direta e indireta pela União de portos e instalações portuárias e sobre as atividades desempenhadas pelos operadores portuários; altera as Leis nºs 5.025, de 10 de junho de 1966, 10.233, de 5 de junho de 2001, 10.683, de 28 de maio de 2003, 9.719, de 27 de novembro de 1998, e 8.213, de 24 de julho de 1991; revoga as Leis nºs 8.630, de 25 de fevereiro de 1993, e 11.610, de 12 de dezembro de 2007, e dispositivos das Leis nºs 11.314, de 3 de julho de 2006, e 11.518, de 5 de setembro de 2007; e dá outras providências.

vote_id: 5376
data_hora_fim: 2013-05-16 02:17:44.0
titulo: DVS - BLOCO PR - PARÁGRAFO ÚNICO DO ART. 59 DO PLV
orientacao_governo: Sim

votos_pro_gov: 8.0
votos_contra_gov: 38.0

url_proposicao: https://dadosa

E ele de fato tende a se dividir nas votações mais polêmicas?

#### Quantas e quais votações superaram uma marca de 2/3, 3/5 ou 1/2 dos parlamentares pró-governo?

In [39]:
def find_surpassing_bills(df, threshold, vote = None ):
    '''
    >> DESCRIÇÃO
    A função recebe um dataframe, um tipo de voto
    ('com_governo' ou 'contra_governo') e uma fração.
    Ela retorna o dataframe filtrado, contendo apenas
    as votações em que o número de votos escolhido
    foi superior ao limite.
    
    
    >> PARÂMETROS
    df        -> O dataframe com os votos que se deseja analisar
    threshold -> O limite que os votos selecionados devem superar
    vote      -> Uma string que determina o tipo de voto a ser 
    avaliado ('com_governo' ou 'contra_governo'. Também é possível
    não passar nenhum valor, acionando o comportamento default, que
    procura qualquer votação superior ao threshold.
    '''
    
    df_ = df.copy()
    df_ = df_[ df_[vote] > 513 * (threshold) ]
    return df_

In [40]:
find_surpassing_bills(df_general, 2/3, 'com_governo').sort_values(by='com_governo', ascending=False)

,voteId,idProposicao,dataHoraFim,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,com_governo_%,contra_governo_%
856,6088,602633,2015-02-10 21:41:41.0,378,NaN,NaN,452.0,20.0,0.957627,0.042373
863,6126,339287,2015-03-10 21:42:20.0,378,NaN,NaN,452.0,3.0,0.993407,0.006593
1149,7291,2080604,2016-11-30 00:12:31.0,325,NaN,NaN,450.0,5.0,0.989011,0.010989
1033,6829,1063052,2016-02-23 21:26:38.0,378,NaN,NaN,448.0,4.0,0.991150,0.008850
1031,6819,611966,2016-02-17 21:52:46.0,378,NaN,NaN,445.0,12.0,0.973742,0.026258
34,1932,113716,2003-08-13 23:08:57.0,120,120.0,120.0,445.0,18.0,0.961123,0.038877
873,6169,339287,2015-03-24 22:04:16.0,378,NaN,NaN,441.0,9.0,0.980000,0.020000
248,2490,142458,2006-01-18 21:33:36.0,136,136.0,NaN,439.0,14.0,0.969095,0.030905
31,1929,113716,2003-08-13 23:08:57.0,120,120.0,120.0,437.0,23.0,0.950000,0.050000
1051,6942,533082,2016-05-03 21:35:55.0,325,NaN,NaN,433.0,11.0,0.975225,0.024775


In [41]:
find_surpassing_bills(df_general, 3/5, 'com_governo').sort_values(by='com_governo', ascending=False)

,voteId,idProposicao,dataHoraFim,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,com_governo_%,contra_governo_%
856,6088,602633,2015-02-10 21:41:41.0,378,NaN,NaN,452.0,20.0,0.957627,0.042373
863,6126,339287,2015-03-10 21:42:20.0,378,NaN,NaN,452.0,3.0,0.993407,0.006593
1149,7291,2080604,2016-11-30 00:12:31.0,325,NaN,NaN,450.0,5.0,0.989011,0.010989
1033,6829,1063052,2016-02-23 21:26:38.0,378,NaN,NaN,448.0,4.0,0.991150,0.008850
34,1932,113716,2003-08-13 23:08:57.0,120,120.0,120.0,445.0,18.0,0.961123,0.038877
1031,6819,611966,2016-02-17 21:52:46.0,378,NaN,NaN,445.0,12.0,0.973742,0.026258
873,6169,339287,2015-03-24 22:04:16.0,378,NaN,NaN,441.0,9.0,0.980000,0.020000
248,2490,142458,2006-01-18 21:33:36.0,136,136.0,NaN,439.0,14.0,0.969095,0.030905
31,1929,113716,2003-08-13 23:08:57.0,120,120.0,120.0,437.0,23.0,0.950000,0.050000
1051,6942,533082,2016-05-03 21:35:55.0,325,NaN,NaN,433.0,11.0,0.975225,0.024775


In [42]:
find_surpassing_bills(df_general, 1/2, 'com_governo').sort_values(by='com_governo', ascending=False)

,voteId,idProposicao,dataHoraFim,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,com_governo,contra_governo,com_governo_%,contra_governo_%
863,6126,339287,2015-03-10 21:42:20.0,378,NaN,NaN,452.0,3.0,0.993407,0.006593
856,6088,602633,2015-02-10 21:41:41.0,378,NaN,NaN,452.0,20.0,0.957627,0.042373
1149,7291,2080604,2016-11-30 00:12:31.0,325,NaN,NaN,450.0,5.0,0.989011,0.010989
1033,6829,1063052,2016-02-23 21:26:38.0,378,NaN,NaN,448.0,4.0,0.991150,0.008850
1031,6819,611966,2016-02-17 21:52:46.0,378,NaN,NaN,445.0,12.0,0.973742,0.026258
34,1932,113716,2003-08-13 23:08:57.0,120,120.0,120.0,445.0,18.0,0.961123,0.038877
873,6169,339287,2015-03-24 22:04:16.0,378,NaN,NaN,441.0,9.0,0.980000,0.020000
248,2490,142458,2006-01-18 21:33:36.0,136,136.0,NaN,439.0,14.0,0.969095,0.030905
31,1929,113716,2003-08-13 23:08:57.0,120,120.0,120.0,437.0,23.0,0.950000,0.050000
1051,6942,533082,2016-05-03 21:35:55.0,325,NaN,NaN,433.0,11.0,0.975225,0.024775


#### Quantas votações avaliamos?

In [43]:
df.shape

(31714, 10)